## Imports

In [ ]:
import sys
sys.path.append('../input/efficientnet-weights/brain_automl/')
sys.path.append('../input/efficientnet-weights/brain_automl/efficientnetv2/')

import effnetv2_model

In [ ]:
import warnings
from multiprocessing import cpu_count
warnings.simplefilter('ignore')

In [ ]:
import tensorflow as tf, re, math
import tensorflow_addons as tfa

from tensorflow.keras import layers, optimizers, losses, metrics, callbacks, initializers
from tensorflow.keras import Sequential, Model, Input

import os
import sys
import glob
import json
import gc

from functools import partial

import random
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

import seaborn as sns

import albumentations as A
import imgaug.augmenters as iaa

import cv2
from PIL import Image

print(f'tensorflow version: {tf.__version__}')
print(f'python version: P{sys.version}')

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)    
    
seed_everything(42)

## TPU&GPU Check

In [ ]:
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
train = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv')
train = train.dropna().reset_index(drop=True)
display(train.head())

## Parameters

In [ ]:
# Input Image Shape
IMG_SIZE = 224
N_CHANNELS = 3
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, N_CHANNELS)
N_SAMPLES = len(train)

N_EPOCHS = 15

BATCH_SIZE_BASE = 16
BATCH_SIZE = BATCH_SIZE_BASE * REPLICAS

# ImageNet Normalization
IMAGENET_MEAN = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
IMAGENET_STD = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)

AUTO = tf.data.experimental.AUTOTUNE
EPS = tf.keras.backend.epsilon()

val_fold = 0
k_fold = 5
N_SPLITS=10

print(f'N_SAMPLES: {N_SAMPLES}, BATCH_SIZE: {BATCH_SIZE}')

## Train CSV

In [ ]:
def get_image_path(id, dir):
    return f"{dir + id}"

train["image_path"] = train["image"].apply(get_image_path, dir='../input/sorghum-id-fgvc-9/train_images/')

encoder = LabelEncoder()
ids2label = {l: i for (i, l) in enumerate(encoder.fit(train["cultivar"]).classes_)}
label2ids = {x[1]: x[0] for x in ids2label.items()}

train["cultivar_id"] = encoder.fit_transform(train["cultivar"])

skf = StratifiedKFold(n_splits=N_SPLITS)
for fold, (_, val_) in enumerate(skf.split(X=train, y=train.cultivar)):
    train.loc[val_, "kfold"] = fold
    
train.head()

## Number of Labels

In [ ]:
N_CULTIVAR = train['cultivar'].nunique()
print(f'N_INDIVIDUAL_IDS: {N_CULTIVAR}')

## Augmentation

In [ ]:
transforms_train = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.Cutout(max_h_size=int(IMG_SIZE * 0.4), max_w_size=int(IMG_SIZE * 0.4), num_holes=1, p=0.5)
    ])

## Dataset Utils

In [ ]:
def decode_image(filename, label=None, image_size=IMG_SIZE):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.image.resize(image, [image_size, image_size])
    image = tf.cast(image, tf.float32) / 255.0
    #image = (image - IMAGENET_MEAN) / IMAGENET_STD
    return image, label
    
def aug_fn(image, img_size=IMG_SIZE):
    data = {"image":image}
    aug_data = transforms_train(**data)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_data(image, label, img_size=IMG_SIZE):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float32)
    return aug_img, label

def set_shapes(image, label, img_shape=INPUT_SHAPE):
    image.set_shape(img_shape)
    #label.set_shape([])
    return image, label

def one_hot(image, label, CLASSES=N_CULTIVAR):
    return image, tf.one_hot(indices=label, 
                             depth=CLASSES,
                             dtype=tf.float32)

def view_image(ds):
    image, label = next(iter(ds)) # extract 1 batch from the dataset
    image = image.numpy()
    label = label.numpy()
    
    fig = plt.figure(figsize=(22, 22))
    for i in range(20):
        ax = fig.add_subplot(4, 5, i+1, xticks=[], yticks=[])
        ax.imshow(image[i])
        ax.set_title(f"Label: {label[i]}")

## Train and Valid Dataset

In [ ]:
train_df = train[train.kfold%k_fold != val_fold].reset_index(drop=True)

val_df = train[train.kfold%k_fold == val_fold].reset_index(drop=True)

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_df.image_path, train_df.cultivar_id))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(partial(process_data, img_size=IMG_SIZE), num_parallel_calls=AUTO)
    .map(set_shapes, num_parallel_calls=AUTO)
    .map(one_hot, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(1024)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((val_df.image_path, val_df.cultivar_id))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(one_hot, num_parallel_calls=AUTO)                        
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

## Model

In [ ]:
def get_model():
    tf.keras.backend.clear_session()
    
    tf.config.optimizer.set_jit(True)

    with strategy.scope():
        image = Input(INPUT_SHAPE, name='image', dtype=tf.float32)
        
        cnn = effnetv2_model.get_model('efficientnet-b5', include_top=False, weights=None)
        WEIGHT_PATH = '../input/efficientnet-weights/noisy_student_efficientnet-b5'
        ckpt = tf.train.latest_checkpoint(WEIGHT_PATH)
        cnn.load_weights(ckpt)
        
        embedding = cnn(image)
        
        outputs = layers.Dropout(0.25)(embedding)
        
        outputs = layers.Dense(N_CULTIVAR, activation="softmax", name='predictions')(outputs)
        
        model = Model(inputs=image, outputs=outputs)
        
        # OPTIMIZER
        #optimizer = optimizers.Adam()
        optimizer = tfa.optimizers.AdamW(weight_decay=1e-8)
        
        # LOSS
        loss = {
            'predictions': losses.CategoricalCrossentropy(label_smoothing=0.1),
        }
 
        # METRICS
        metricsx =[
            tfa.metrics.F1Score(num_classes=N_CULTIVAR, name='f1_score'),
            metrics.TopKCategoricalAccuracy(k=1, name='top1acc'),
        ]

        # Compile Model
        model.compile(optimizer=optimizer, loss=loss, metrics=metricsx)

        return model

In [ ]:
model = get_model()

In [ ]:
model.summary()

## Learning Rate Scheduler

In [ ]:
class CosineScheduler():
    def __init__(self, max_update, base_lr=0.01, final_lr=0,
               warmup_steps=0, warmup_begin_lr=0):
        self.base_lr_orig = base_lr
        self.max_update = max_update
        self.final_lr = final_lr
        self.warmup_steps = warmup_steps
        self.warmup_begin_lr = warmup_begin_lr
        self.max_steps = self.max_update - self.warmup_steps

    def get_warmup_lr(self, epoch):
        increase = (self.base_lr_orig - self.warmup_begin_lr) \
                       * float(epoch) / float(self.warmup_steps)
        return self.warmup_begin_lr + increase

    def __call__(self, epoch):
        if epoch < self.warmup_steps:
            return self.get_warmup_lr(epoch)
        if epoch <= self.max_update:
            self.base_lr = self.final_lr + (
                self.base_lr_orig - self.final_lr) * (1 + math.cos(
                math.pi * (epoch - self.warmup_steps) / self.max_steps)) / 2
        return self.base_lr

scheduler = CosineScheduler(max_update=N_EPOCHS, base_lr=1e-3, final_lr=1e-4, warmup_steps=3, warmup_begin_lr=1e-4)
plt.plot([scheduler(t) for t in range(N_EPOCHS)])
plt.show()

## Callbacks

In [ ]:
model_checkpoint_callback = callbacks.ModelCheckpoint(
    'model_best.h5', 
    monitor='loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True
)

model_checkpoint_callback.set_model(model)

learning_rate_callback = callbacks.LearningRateScheduler(scheduler, verbose=1)

## Train

In [ ]:
STEPS_PER_EPOCH = len(train_df) // BATCH_SIZE

STEPS_PER_EPOCH

In [ ]:
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=N_EPOCHS,
                    callbacks=[learning_rate_callback,
                              model_checkpoint_callback,],
                    steps_per_epoch=STEPS_PER_EPOCH
                   )

## Training History


In [ ]:
def plot_history_metric(metric, f_best=np.argmax, yscale='linear'):
    x = np.arange(1, len(history.history[metric]) + 1)
    y_train = history.history[metric]
    plt.figure(figsize=(20, 8))
    # TRAIN
    plt.plot(x, y_train, color='tab:blue', lw=3, label='train')
    plt.title(f'Training {metric}', fontsize=24, pad=10)
    plt.ylabel(metric, fontsize=20, labelpad=10)
    plt.xlabel('epoch', fontsize=20, labelpad=10)
    plt.xticks([1] + np.arange(5, N_EPOCHS + 1, 5).tolist(), fontsize=16) # set tick step to 1 and let x axis start at 1
    plt.yticks(fontsize=16)
    plt.yscale(yscale)
    
    # Train Best Marker
    x_best = f_best(y_train)
    y_best = y_train[x_best]
    plt.scatter(x_best + 1, y_best, color='purple', s=100, marker='o', label=f'train best: {y_best:.4f}')
 
    if f'val_{metric}' in history.history:
        y_val = history.history[f'val_{metric}']
       # Validation Best Marker
        plt.plot(x, y_val, color='tab:orange', lw=3, label='validation')
        # VALIDATION
        x_best = f_best(y_val)
        y_best = y_val[x_best]
        plt.scatter(x_best + 1, y_best, color='red', s=100, marker='o', label=f'validation best: {y_best:.4f}')
    
    plt.grid()
    plt.legend(prop={'size': 18})
    plt.show()

In [ ]:
plot_history_metric('loss', f_best=np.argmin)

In [ ]:
plot_history_metric('top1acc', f_best=np.argmax)

## Prediction

In [ ]:
model.load_weights('./model_best.h5')

@tf.function(experimental_compile=True)
def predict_on_batch(images):
    return model(images, training=False)

In [ ]:
test_df = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv')

test_df["image_path"] = test_df["filename"].apply(get_image_path, dir='../input/sorghum-id-fgvc-9/test/')

test_df.head()

In [ ]:
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((test_df.image_path, test_df.filename))
    .map(decode_image, num_parallel_calls=AUTO)                 
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [ ]:
N_TEST_SAMPLES = len(glob.glob('../input/sorghum-id-fgvc-9/test/*png'))
TEST_EMBEDDINGS = np.zeros(shape=[N_TEST_SAMPLES, N_CULTIVAR], dtype=np.float32)
TEST_EMBEDDINGS_IDS = np.empty(shape=[N_TEST_SAMPLES], dtype=object)

total = math.ceil(N_TEST_SAMPLES / BATCH_SIZE)
offset = 0

with strategy.scope():
    for idx, (images, image_ids) in enumerate(tqdm(test_dataset, total=total)):
        start = offset
        end = offset + len(images)
        TEST_EMBEDDINGS[start:end] = predict_on_batch(images)
        TEST_EMBEDDINGS_IDS[start:end] = image_ids.numpy().astype(str)
        offset += len(images)

In [ ]:
print(f'TEST_EMBEDDINGS shape: {TEST_EMBEDDINGS.shape}, dtype: {TEST_EMBEDDINGS.dtype}')
print(f'TEST_EMBEDDINGS_IDS shape: {TEST_EMBEDDINGS_IDS.shape}, dtype: {TEST_EMBEDDINGS_IDS.dtype}')

In [ ]:
TEST_EMBEDDINGS_TF = tf.constant(TEST_EMBEDDINGS)
TEST_EMBEDDINGS_IDS_TF = tf.constant(TEST_EMBEDDINGS_IDS)

In [ ]:
del TEST_EMBEDDINGS, TEST_EMBEDDINGS_IDS
gc.collect()

## Submission

In [ ]:
pred = tf.argmax(TEST_EMBEDDINGS_TF ,axis=1)
predicts = [label2ids[x] for x in pred.numpy()]
files = [i.decode() for i in TEST_EMBEDDINGS_IDS_TF.numpy()]

sub = pd.DataFrame(zip(files, predicts), columns=['filename', 'cultivar'])

sub.to_csv('submission.csv', index=False)